# Reference frames

rigid_body_motion provides a flexible high-performance framework for working offline with motion data. The core of this framework is a mechanism for constructing trees of both static and dynamic reference frames that supports automatic lookup and application of transformations across the tree.

In [ ]:
import numpy as np
import rigid_body_motion as rbm

<div class="alert alert-info">
Note
    
The following examples require the `matplotlib` library.
</div>

In [ ]:
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (6, 6)

## Static frames

We will begin by defining a world reference frame using the [ReferenceFrame](_generated/rigid_body_motion.ReferenceFrame.rst) class:

In [ ]:
rf_world = rbm.ReferenceFrame("world")

Now we can add a second reference frame as a child of the world frame. This frame is translated by 5 meters (or any other unit) in the x direction and rotated 180° around the yaw axis. Note that rotations are represented as [unit quaternions](https://en.wikipedia.org/wiki/Quaternions_and_spatial_rotation) by default.

In [ ]:
rf_observer = rbm.ReferenceFrame(
    "observer", parent=rf_world, translation=(5, 0, 0), rotation=(0, 0, 0, 1)
)

We can show the reference frame tree with the [render_tree](_generated/rigid_body_motion.render_tree.rst) function:

In [ ]:
rbm.render_tree(rf_world)

It is also possible to show a 3d plot of static reference frames with [plot.reference_frame()](_generated/rigid_body_motion.plot.reference_frame.rst):

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection="3d")

rbm.plot.reference_frame(rf_world, ax=ax)
rbm.plot.reference_frame(rf_observer, rf_world, ax=ax)

To facilitate referring to previously defined frames, the library has a registry where frames can be stored (and later looked up) by name with [ReferenceFrame.register()](api/_generated/rigid_body_motion.ReferenceFrame.register.rst):

In [ ]:
rf_world.register()
rf_observer.register()
rbm.registry

## Transforming positions and orientations

Now that we've set up a basic tree, we can use it to transform motion between reference frames. Let's assume we have a body that moves from the origin of the world frame to the origin of the observer frame in 5 steps:

In [ ]:
p_body_world = np.zeros((5, 3))
p_body_world[:, 0] = np.linspace(0, 5, 5)
p_body_world[:, 0]

We can add the body positions to the plot with [plot.points()](_generated/rigid_body_motion.plot.points.rst):

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection="3d")

rbm.plot.reference_frame(rf_world, ax=ax)
rbm.plot.reference_frame(rf_observer, rf_world, ax=ax)
rbm.plot.points(p_body_world, ax=ax, fmt="yo-")

To compute the position of the body with respect to (wrt) the observer, we use [transform_points()](_generated/rigid_body_motion.transform_points.rst):

In [ ]:
p_body_observer = rbm.transform_points(p_body_world, outof="world", into="observer")
p_body_observer[:, 0]

As expected, the resulting motion of the body is in the x direction, towards the observer.

Orientations expressed in quaternions can be transformed with [transform_quaternions()](_generated/rigid_body_motion.transform_quaternions.rst):

In [ ]:
orientation = [1, 0, 0, 0]
rbm.transform_quaternions(orientation, outof="world", into="observer")

## Moving frames

In [ ]:
ts_body = np.arange(5)

In [ ]:
rbm.register_frame("body", translation=p_body_world, timestamps=ts_body, parent="world")
rbm.render_tree("world")

In [ ]:
rbm.transform_points([1, 0, 0], outof="world", into="body")[:, 0]

In [ ]:
p_body2_world = np.zeros((5, 3))
p_body2_world[:, 1] = np.linspace(0, 1, 5)
ts_body2 = ts_body - 0.5

In [ ]:
p_body2_body, ts_body2_body = rbm.transform_points(
    p_body2_world ,
    outof="world",
    into="body",
    timestamps=ts_body2,
    return_timestamps=True,
)

Let's plot the position of both bodies wrt the world frame as well as the position of the second body wrt the first body to see how the timestamp matching works:

In [ ]:
fig, axarr = plt.subplots(3, 1, sharex=True, sharey=True)

axarr[0].plot(ts_body, p_body_world, "*-")
axarr[0].legend(["x", "y", "z"])
axarr[0].set_ylabel("Position (m)")
axarr[0].set_title("First body wrt world frame")
axarr[0].grid("on")

axarr[1].plot(ts_body2, p_body2_world, "*-")
axarr[1].set_ylabel("Position (m)")
axarr[1].set_title("Second body wrt world frame")
axarr[1].grid("on")

axarr[2].plot(ts_body2_body, p_body2_body, "*-")
axarr[2].set_xlabel("Time (s)")
axarr[2].set_ylabel("Position (m)")
axarr[2].set_title("Second body wrt first body frame")
axarr[2].grid("on")

fig.tight_layout()

## Looking up transforms and velocities

In [ ]:
translation, rotation, ts = rbm.lookup_transform("body", "world")
translation[:, 0]

In [ ]:
linear, angular, ts = rbm.lookup_twist("body", "world")
linear[:, 0]